In [1]:
import os
import sys
import tensorflow as tf
import numpy as np
import math
import timeit
import scipy

#import matplotlib.pyplot as plt
import CNet
import provider
import TR
#%matplotlib inline

In [2]:
USE_GPU = True

if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

# Constant to control how often we print when training models
print_every = 100

print('Using device: ', device)

Using device:  /device:GPU:0


In [4]:
BATCH_SIZE=8
MAX_EPOCH=1
learning_rate = 3e-5
channel_1, channel_2, num_classes,c1 = 32, 16, 2, 4

def model_init_fn(inputs1,inputs2, is_training):
    model = None

    model = CNet.CentralNet(channel_1, channel_2, num_classes,c1)

    return model(inputs1,inputs2)

def optimizer_init_fn():
    optimizer = None

    optimizer = tf.train.GradientDescentOptimizer(learning_rate)

    return optimizer

TR.train_part34(model_init_fn, optimizer_init_fn,20,device,BATCH_SIZE)

['data/demo/DVR/2009.jpg' 'data/demo/DVR/539.jpg' 'data/demo/DVR/2872.jpg'
 ... 'data/demo/DVR/1743.jpg' 'data/demo/DVR/824.jpg'
 'data/demo/DVR/2427.jpg']
Starting epoch 0
mean loss: 42.197952
accuracy (angle): 0.151786
accuracy (speed): 0.265542
eval mean loss: 0.416786
eval accuracy (angle): 0.174202
eval accuracy (speed): 0.390957
Starting epoch 1
mean loss: 0.430768
accuracy (angle): 0.231481
accuracy (speed): 0.427910
eval mean loss: 0.399458
eval accuracy (angle): 0.281915
eval accuracy (speed): 0.444149
Starting epoch 2
mean loss: 0.399258
accuracy (angle): 0.266534
accuracy (speed): 0.475529
eval mean loss: 0.331830
eval accuracy (angle): 0.303191
eval accuracy (speed): 0.486702
Starting epoch 3
mean loss: 0.390691
accuracy (angle): 0.291667
accuracy (speed): 0.483796
eval mean loss: 0.326351
eval accuracy (angle): 0.315160
eval accuracy (speed): 0.468085
Starting epoch 4
mean loss: 0.361423
accuracy (angle): 0.304563
accuracy (speed): 0.504630
eval mean loss: 0.332202
eval ac